In [ ]:
%pip install boto3=='1.28.62'

In [17]:
from abc import ABC
import boto3
import json

In [18]:
import os
#os.environ["AWS_ACCESS_KEY_ID"]=""
#os.environ["AWS_SECRET_ACCESS_KEY"]=""
#os.environ["AWS_DEFAULT_REGION"]=""

In [19]:
class Prompt(ABC):
    def __init__(self, speaker, text):
        self.speaker = speaker
        self.text = text

    def __str__(self):
        return f"\n\n{self.speaker}: {self.text}"

    def __repr__(self):
        return self.__str__()

In [20]:
class HumanPrompt(Prompt):
    def __init__(self, text):
        super().__init__("Human", text)

In [21]:
class AssistantPrompt(Prompt):
    def __init__(self, text):
        super().__init__("Assistant", text)

In [22]:
class ConversationChain:
    def __init__(self, prompts: list = None, generator = None):
        if prompts is None:
            prompts = []
        self.prompts = prompts
        self.generator = generator

    def __str__(self):
        return "\n".join([str(p) for p in self.prompts])

    def __repr__(self):
        return self.__str__()

    def __add__(self, prompt: Prompt):
        return ConversationChain(self.prompts + [prompt], self.generator)

    # access list elements with []
    def __getitem__(self, index): #-> Prompt | ConversationChain:
        print("Getting item for index", index)
        if isinstance(index, slice):
            return ConversationChain(self.prompts[index], self.generator)
        else:
            return self.prompts[index]

    def generate(self):
        # non mutating self
        prompt = self.__str__() + "\n\Assistant:"
        # print(prompt)
        generated = ''.join(self.generator(prompt))
        return self + AssistantPrompt(generated)

In [23]:
def invoke_bedrock_anthropic_claude_v2(prompt):
    """Invoke Anthropic Claude v2 model."""
    bedrock_runtime = boto3.client(
        service_name='bedrock-runtime',
        region_name='us-east-1',
        endpoint_url='https://bedrock.us-east-1.amazonaws.com')
    def body(_prompt: str) -> str:
        return json.dumps(
            {
                'prompt': _prompt,
                'max_tokens_to_sample':2048,
                'temperature':0.0,
                'top_k':250,
                'top_p':0.999,
                'stop_sequences':['Human:'],
            },
        )
    model_id = 'anthropic.claude-v2'
    accept = '*/*'
    content_type = 'application/json'
    try:
        response = bedrock_runtime.invoke_model_with_response_stream(
            body=body(prompt),
            modelId=model_id,
            accept=accept,
            contentType=content_type)

        request_id = response['ResponseMetadata']['RequestId']

        for m in response['body']:
            b = m.get('chunk', {}).get('bytes', b'')
            json_response: dict = json.loads(b)
            resp: str = json_response.get('completion', '')
            # print('🤖 Out <%s>: %s', request_id, repr(resp))
            yield resp
            # relinquish control to the event loop,
            # allowing other tasks and callbacks to run

            if json_response['stop_reason']:
                # print(
                #     '🤖 Stop <%s> with %s',
                #     request_id,
                #     json_response['stop_reason'],
                # )
                break

    except Exception as err:
        raise err


In [24]:
ingredients = "sugar,  flour, cocoa powder,chocolate syrup"
restrictions = "should'nt be too sweet, should have low calories"
diet_preferences = "italian"

In [25]:
prompt = "Can you please recommend me 5 vegan recipes with these ingredients - "+ingredients+". I also have these restrictions - "+ restrictions+ ". Along with these preferences - "+ diet_preferences
prompt

"Can you please recommend me 5 vegan recipes with these ingredients - sugar,  flour, cocoa powder,chocolate syrup. I also have these restrictions - should'nt be too sweet, should have low calories. Along with these preferences - italian"

In [26]:
s1  = HumanPrompt(prompt)

chain = ConversationChain([s1], generator=invoke_bedrock_anthropic_claude_v2)
print(chain)



Human: Can you please recommend me 5 vegan recipes with these ingredients - sugar,  flour, cocoa powder,chocolate syrup. I also have these restrictions - should'nt be too sweet, should have low calories. Along with these preferences - italian


In [27]:
chain = chain.generate()
chain



Human: Can you please recommend me 5 vegan recipes with these ingredients - sugar,  flour, cocoa powder,chocolate syrup. I also have these restrictions - should'nt be too sweet, should have low calories. Along with these preferences - italian


Assistant:  Here are 5 vegan recipe ideas using sugar, flour, cocoa powder, and chocolate syrup that aren't too sweet, are lower in calories, and have an Italian flair:

1. Vegan Chocolate Hazelnut Biscotti - Make biscotti dough with flour, sugar, cocoa powder, chopped hazelnuts. Shape into a log, bake, slice, and bake again. Dip in melted chocolate or chocolate-hazelnut spread.

2. Vegan Chocolate Amaretti Cookies - Mix almond flour, sugar, cocoa powder, almond extract to make small cookies. Bake and dust with powdered sugar.

3. Vegan Chocolate Espresso Pudding - Mix cocoa powder, sugar, cornstarch, non-dairy milk, espresso powder. Cook on the stove until thickened. Chill and top with whipped coconut cream. 

4. Vegan Chocolate Olive Oil Cak

In [28]:
chain1 = ConversationChain([chain], generator=invoke_bedrock_anthropic_claude_v2)
chain2 = ConversationChain([chain], generator=invoke_bedrock_anthropic_claude_v2)
chain3 = ConversationChain([chain], generator=invoke_bedrock_anthropic_claude_v2)
chain4 = ConversationChain([chain], generator=invoke_bedrock_anthropic_claude_v2)
chain5 = ConversationChain([chain], generator=invoke_bedrock_anthropic_claude_v2)

In [29]:
chain1 = chain + \
    HumanPrompt("Ok now generate the entire list of things to do for the first recipe, the quantities of each ingredients, the proportions and the procedure in steps.")+ \
chain1
chain1 = chain1.generate()
chain1



Human: Can you please recommend me 5 vegan recipes with these ingredients - sugar,  flour, cocoa powder,chocolate syrup. I also have these restrictions - should'nt be too sweet, should have low calories. Along with these preferences - italian


Assistant:  Here are 5 vegan recipe ideas using sugar, flour, cocoa powder, and chocolate syrup that aren't too sweet, are lower in calories, and have an Italian flair:

1. Vegan Chocolate Hazelnut Biscotti - Make biscotti dough with flour, sugar, cocoa powder, chopped hazelnuts. Shape into a log, bake, slice, and bake again. Dip in melted chocolate or chocolate-hazelnut spread.

2. Vegan Chocolate Amaretti Cookies - Mix almond flour, sugar, cocoa powder, almond extract to make small cookies. Bake and dust with powdered sugar.

3. Vegan Chocolate Espresso Pudding - Mix cocoa powder, sugar, cornstarch, non-dairy milk, espresso powder. Cook on the stove until thickened. Chill and top with whipped coconut cream. 

4. Vegan Chocolate Olive Oil Cak

In [30]:
chain2 = chain + \
    HumanPrompt("Ok now generate the entire list of things to do for the second recipe, the quantities of each ingredients, the proportions and the procedure in steps.")+ \
chain2
chain2 = chain2.generate()
chain2



Human: Can you please recommend me 5 vegan recipes with these ingredients - sugar,  flour, cocoa powder,chocolate syrup. I also have these restrictions - should'nt be too sweet, should have low calories. Along with these preferences - italian


Assistant:  Here are 5 vegan recipe ideas using sugar, flour, cocoa powder, and chocolate syrup that aren't too sweet, are lower in calories, and have an Italian flair:

1. Vegan Chocolate Hazelnut Biscotti - Make biscotti dough with flour, sugar, cocoa powder, chopped hazelnuts. Shape into a log, bake, slice, and bake again. Dip in melted chocolate or chocolate-hazelnut spread.

2. Vegan Chocolate Amaretti Cookies - Mix almond flour, sugar, cocoa powder, almond extract to make small cookies. Bake and dust with powdered sugar.

3. Vegan Chocolate Espresso Pudding - Mix cocoa powder, sugar, cornstarch, non-dairy milk, espresso powder. Cook on the stove until thickened. Chill and top with whipped coconut cream. 

4. Vegan Chocolate Olive Oil Cak

In [31]:
chain3 = chain + \
    HumanPrompt("Ok now generate the entire list of things to do for the third recipe, the quantities of each ingredients, the proportions and the procedure in steps.")+ \
chain3
chain3 = chain3.generate()
chain3



Human: Can you please recommend me 5 vegan recipes with these ingredients - sugar,  flour, cocoa powder,chocolate syrup. I also have these restrictions - should'nt be too sweet, should have low calories. Along with these preferences - italian


Assistant:  Here are 5 vegan recipe ideas using sugar, flour, cocoa powder, and chocolate syrup that aren't too sweet, are lower in calories, and have an Italian flair:

1. Vegan Chocolate Hazelnut Biscotti - Make biscotti dough with flour, sugar, cocoa powder, chopped hazelnuts. Shape into a log, bake, slice, and bake again. Dip in melted chocolate or chocolate-hazelnut spread.

2. Vegan Chocolate Amaretti Cookies - Mix almond flour, sugar, cocoa powder, almond extract to make small cookies. Bake and dust with powdered sugar.

3. Vegan Chocolate Espresso Pudding - Mix cocoa powder, sugar, cornstarch, non-dairy milk, espresso powder. Cook on the stove until thickened. Chill and top with whipped coconut cream. 

4. Vegan Chocolate Olive Oil Cak

In [32]:
chain3 = chain + \
    HumanPrompt("Ok now generate the entire list of things to do for the third recipe, the quantities of each ingredients, the proportions and the procedure in steps.")+ \
chain3
chain3 = chain3.generate()
chain3



Human: Can you please recommend me 5 vegan recipes with these ingredients - sugar,  flour, cocoa powder,chocolate syrup. I also have these restrictions - should'nt be too sweet, should have low calories. Along with these preferences - italian


Assistant:  Here are 5 vegan recipe ideas using sugar, flour, cocoa powder, and chocolate syrup that aren't too sweet, are lower in calories, and have an Italian flair:

1. Vegan Chocolate Hazelnut Biscotti - Make biscotti dough with flour, sugar, cocoa powder, chopped hazelnuts. Shape into a log, bake, slice, and bake again. Dip in melted chocolate or chocolate-hazelnut spread.

2. Vegan Chocolate Amaretti Cookies - Mix almond flour, sugar, cocoa powder, almond extract to make small cookies. Bake and dust with powdered sugar.

3. Vegan Chocolate Espresso Pudding - Mix cocoa powder, sugar, cornstarch, non-dairy milk, espresso powder. Cook on the stove until thickened. Chill and top with whipped coconut cream. 

4. Vegan Chocolate Olive Oil Cak

In [33]:
chain5 = chain + \
    HumanPrompt("Ok now generate the entire list of things to do for the fifth recipe, the quantities of each ingredients, the proportions and the procedure in steps.")+ \
chain5
chain5 = chain5.generate()
chain5



Human: Can you please recommend me 5 vegan recipes with these ingredients - sugar,  flour, cocoa powder,chocolate syrup. I also have these restrictions - should'nt be too sweet, should have low calories. Along with these preferences - italian


Assistant:  Here are 5 vegan recipe ideas using sugar, flour, cocoa powder, and chocolate syrup that aren't too sweet, are lower in calories, and have an Italian flair:

1. Vegan Chocolate Hazelnut Biscotti - Make biscotti dough with flour, sugar, cocoa powder, chopped hazelnuts. Shape into a log, bake, slice, and bake again. Dip in melted chocolate or chocolate-hazelnut spread.

2. Vegan Chocolate Amaretti Cookies - Mix almond flour, sugar, cocoa powder, almond extract to make small cookies. Bake and dust with powdered sugar.

3. Vegan Chocolate Espresso Pudding - Mix cocoa powder, sugar, cornstarch, non-dairy milk, espresso powder. Cook on the stove until thickened. Chill and top with whipped coconut cream. 

4. Vegan Chocolate Olive Oil Cak

In [34]:
chain1[-1]

Getting item for index -1




Assistant:  Here are the detailed instructions for the Vegan Chocolate Hazelnut Biscotti recipe:

Ingredients:
- 2 cups all-purpose flour 
- 1/2 cup sugar
- 1/3 cup unsweetened cocoa powder
- 1 tsp baking powder
- 1/4 tsp salt
- 1/2 cup hazelnuts, chopped  
- 1/3 cup non-dairy milk
- 1 tsp vanilla extract
- 2 tbsp olive oil

Instructions:

1. Preheat oven to 350°F. Line a baking sheet with parchment paper. 

2. In a large bowl, whisk together the flour, sugar, cocoa powder, baking powder and salt. 

3. Add the hazelnuts, non-dairy milk, vanilla and olive oil. Stir until a dough forms. 

4. On a lightly floured surface, shape the dough into a log about 12 inches long and 2 inches wide. 

5. Place log on the prepared baking sheet. Bake for 30 minutes.

6. Remove from oven and let cool for 10 minutes. Reduce oven temperature to 300°F.

7. Using a serrated knife, slice the log diagonally into 1/2 inch thick slices. 

8. Arrange slices cut-side down on the baking sheet. Bake for 10 minute

In [35]:
chain2[-1]

Getting item for index -1




Assistant:  Here are the details for the Vegan Chocolate Amaretti Cookies recipe:

Ingredients:
- 1 1/2 cups almond flour 
- 1/3 cup sugar
- 3 tbsp cocoa powder 
- 1 tsp almond extract
- Pinch of salt

Directions:

1. Preheat oven to 300°F. Line a baking sheet with parchment paper. 

2. In a medium bowl, stir together the almond flour, sugar, cocoa powder, almond extract and salt until well combined. 

3. Roll dough into 1-inch balls and place them about 1 inch apart on the prepared baking sheet. 

4. Bake for 18-20 minutes, until the cookies look cracked on top. Let cool on the baking sheet for 5 minutes before transferring to a wire rack to cool completely.

5. Once completely cooled, dust the cookies lightly with powdered sugar.

Makes about 18 small cookies.

Let me know if you need any clarification or have additional questions!

In [36]:
chain3[-1]

Getting item for index -1




Assistant:  Here is the detailed recipe for the Vegan Chocolate Espresso Pudding:

Ingredients:
- 1/3 cup cocoa powder 
- 1/2 cup sugar
- 3 tablespoons cornstarch
- 2 cups non-dairy milk (like almond, soy or oat milk)
- 1 tablespoon instant espresso powder
- 1 teaspoon vanilla extract
- Pinch of salt

For Serving:
- Whipped coconut cream 
- Shaved chocolate 
- Cocoa powder for dusting

Instructions:

1. In a small bowl, whisk together the cocoa powder, sugar, cornstarch, espresso powder and salt. Set aside.

2. In a medium saucepan, heat the non-dairy milk over medium heat until just simmering. 

3. Slowly whisk the cocoa powder mixture into the hot milk. 

4. Bring to a boil, while continuously whisking. Boil for 2 minutes, whisking constantly, until thickened to a pudding consistency.

5. Remove from heat and stir in the vanilla extract. 

6. Pour pudding into a bowl or individual ramekins. Press plastic wrap directly on surface of pudding to prevent skin forming. Refrigerate for a

In [37]:
chain4[-1]

Getting item for index -1




Assistant:  Here are the details for the Vegan Chocolate Olive Oil Cake recipe:

Ingredients:
- 1 1/2 cups all-purpose flour 
- 1/2 cup cocoa powder
- 1 1/2 teaspoons baking powder
- 1/2 teaspoon salt
- 1/2 cup granulated sugar
- 1/3 cup olive oil
- 1 teaspoon vanilla extract
- 1 cup non-dairy milk (like almond or soy)
- Powdered sugar for dusting

Instructions:

1. Preheat oven to 350°F. Grease a 9-inch round cake pan with olive oil or non-stick spray.

2. In a medium bowl, whisk together the flour, cocoa powder, baking powder and salt. 

3. In a large bowl, beat the granulated sugar, olive oil, vanilla and non-dairy milk together until well combined. 

4. Slowly beat in the dry ingredients until just combined (do not overmix).

5. Pour batter into prepared cake pan and bake for 30-35 minutes, until a toothpick inserted in the center comes out clean.

6. Allow cake to cool completely before removing from pan. Dust with powdered sugar before serving.

Let me know if you need any clar

In [38]:
chain5[-1]

Getting item for index -1




Assistant:  Here is the detailed recipe for the Vegan Chocolate Biscotti Smoothie:

Ingredients:
- 1 medium banana, frozen
- 1 cup non-dairy milk (almond, soy, oat, etc.) 
- 2 tbsp cocoa powder
- 1/4 cup crushed vegan chocolate biscotti 
- 2 tbsp chocolate syrup
- 2 tbsp coconut cream (for whipping)
- 2-3 vegan chocolate biscotti, for garnish

Instructions:

1. Add the frozen banana, non-dairy milk, cocoa powder, crushed biscotti, and chocolate syrup to a high-powered blender. Blend on high until smooth and creamy.

2. In a small bowl, whip the coconut cream with an electric mixer until light and fluffy, about 2-3 minutes. 

3. Pour the smoothie into a glass. Top with the whipped coconut cream and garnish with biscotti pieces.

4. Enjoy immediately! Makes 1 serving.

Let me know if you need any clarification on quantities or preparation steps. I'm happy to provide more details.